In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,classification_report
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, RobustScaler
from sklearn.ensemble import RandomForestClassifier


df_model = pd.read_csv('loan_project/SBAnational.csv')

/tmp/ipykernel_7125/193558196.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_7125/193558196.py:15: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_model = pd.read_csv('loan_project/SBAnational.csv')


In [2]:
df_model = df_model.dropna()
df_model = df_model.drop(['Name','City','Bank','RevLineCr','LowDoc','ApprovalDate','CreateJob','RetainedJob','ChgOffDate', 'DisbursementDate','DisbursementGross','BalanceGross','ChgOffPrinGr','SBA_Appv'],axis=1)
df_model['GrAppv'] = pd.to_numeric(df_model['GrAppv'].str.replace('[\$,]', '', regex=True))
df_model

,LoanNr_ChkDgt,State,Zip,BankState,NAICS,ApprovalFY,Term,NoEmp,NewExist,FranchiseCode,UrbanRural,MIS_Status,GrAppv
6,1000093009,NJ,7083,SD,0,1980,45,45,2.0,0,0,CHGOFF,600000.0
36,1000554001,OH,44428,OH,0,1997,137,2,1.0,1,0,CHGOFF,47000.0
43,1000653000,IN,46124,IN,0,1980,120,16,2.0,0,0,CHGOFF,200000.0
58,1000726007,ME,4927,SD,236115,2006,167,9,1.0,1,1,CHGOFF,1350000.0
60,1000735003,MI,49417,SD,713930,1997,7,10,1.0,1,0,CHGOFF,25000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
899131,9995033006,TN,37416,TN,233210,1997,6,3,1.0,1,0,CHGOFF,125000.0
899138,9995143004,KS,66212,KS,0,1997,42,9,1.0,54100,0,CHGOFF,100000.0
899143,9995213001,NJ,8053,NJ,235610,1997,62,1,2.0,1,0,CHGOFF,150000.0
899153,9995413002,NY,11797,NY,0,1997,119,5,1.0,1,0,CHGOFF,142000.0


In [3]:
df_model.isnull().sum()

LoanNr_ChkDgt    0
State            0
Zip              0
BankState        0
NAICS            0
ApprovalFY       0
Term             0
NoEmp            0
NewExist         0
FranchiseCode    0
UrbanRural       0
MIS_Status       0
GrAppv           0
dtype: int64

In [4]:
# colonne_categorie = ['LowDoc','RevLineCr']

# tranformer = ColumnTransformer(
#     transformers=[
#         ('impute_colonne_c',SimpleImputer(strategy='constant', fill_value="Absent"),colonne_categorie)
        
#     ],remainder='passthrough')

# my_pipeline = make_pipeline(tranformer)

# my_pipeline.fit(df_model)

# df_model = my_pipeline.transform(df_model)

In [5]:
df_model['ApprovalFY'] = pd.to_numeric(df_model['ApprovalFY'], errors='coerce').astype('Int64')
df_model = df_model.dropna()


X = df_model.drop('MIS_Status', axis=1)
y = df_model.MIS_Status

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42)

num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

my_final_pipeline = make_pipeline(preprocessing)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)

model = make_pipeline(
    my_final_pipeline,
    RandomForestClassifier(random_state=42,max_depth=3)
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)


report = classification_report(y_test, y_pred)

print(report)

# faire une feature importance

/home/utilisateur/Documents/Projets/US_SBA/MLProject/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/utilisateur/Documents/Projets/US_SBA/MLProject/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      CHGOFF       0.97      1.00      0.98     23380
       P I F       0.00      0.00      0.00       749

    accuracy                           0.97     24129
   macro avg       0.48      0.50      0.49     24129
weighted avg       0.94      0.97      0.95     24129



/home/utilisateur/Documents/Projets/US_SBA/MLProject/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
